# Методы машинного обучения – Лабораторная работа №3

# Нелинейная регрессия

Импортируем необходимые библиотеки:

In [ ]:
!pip install -q tfds-nightly

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

### Моделирование градиентного спуска

Градиентный спуск — метод нахождения локального минимума или максимума функции с помощью движения вдоль градиента. 

In [ ]:
plot_x = np.linspace(-1., 6., 141)
plot_y = (plot_x-2.5)**2 - 1.
plt.plot(plot_x, plot_y);

Для вычисления значений и производной функции $y = (x-2.5)^2-1$ будем использовать следующие функции:

In [ ]:
def J(x_):
    return (x_-2.5)**2 - 1.

def dJ(x_):
    return 2*(x_-2.5)

Для моделирования градиентного спуска будем использовать функцию `gradient_descent`, которая будет запоминать и возвращать историю итераций, для визуализации градиентного спуска будем использовать функцию `plot_history`:

In [ ]:
def gradient_descent(initial_x, eta, n_iters = 1e4, epsilon=1e-8):
    x_ = initial_x
    x_history = [initial_x]
    i_iter = 0

    while i_iter < n_iters:
        gradient = dJ(x_)
        last_x_ = x_
        x_ -= eta * gradient
        x_history.append(x_)
    
        if(abs(J(x_) - J(last_x_)) < epsilon):
            break
        i_iter += 1
        
    return x_history
            
def plot_history(plot_x, x_history):
    plt.plot(plot_x, J(plot_x))
    plt.plot(np.array(x_history), J(np.array(x_history)), color="r", marker='+')
    plt.text(1., 10., f'Кол-во шагов: {len(x_history)}', fontsize=14, color='r')

Проведем моделирование градиентного спуска с различным шагом:

In [ ]:
hist = gradient_descent(0., 0.1)
plot_history(plot_x, hist)

In [ ]:
hist = gradient_descent(0., 0.01)
plot_history(plot_x, hist)

In [ ]:
hist = gradient_descent(0., 0.001)
plot_history(plot_x, hist)

In [ ]:
hist = gradient_descent(0., 0.8)
plot_history(plot_x, hist)

При дальнейшем увеличении шага возникает ошибка, которую нужно обработать:

In [ ]:
try:
    hist = gradient_descent(0., 1.1)
    plot_history(plot_x, hist)
except Exception as e: 
    print(f"{type(e).__name__}: {e}")

Ограничимся десятью итерациями:

In [ ]:
hist = gradient_descent(0., 1.1, n_iters=10)
plot_history(plot_x, hist)

### Стохастический градиентный спуск

Стохастический градиентный спуск (stochastic gradient descent, SGD) − оптимизационный алгоритм, отличающийся от обычного градиентного спуска тем, что градиент оптимизируемой функции считается на каждом шаге не как сумма градиентов от каждого элемента выборки, а как градиент от одного, случайно выбранного элемента или некоторой подвыборки.

In [ ]:
m = 100000

x = np.random.normal(size=m)
X = x.reshape(-1,1)           # преобразуем вектор в матрицу с одним столбцом
y = 4.*x + 3. + np.random.normal(0, 3, size=m)

plt.scatter(x, y);

Определим класс `RegressionSGD`, использующий стохастический градиентный спуск (с переменным шагом) при обучении модели:

In [ ]:
class RegressionSGD:

    def __init__(self):
        self.coef_ = None
        self.intercept_ = None
        self._theta = None

    def fit(self, X_train, y_train, n_iters=50, t0=5, t1=50):
        assert X_train.shape[0] == y_train.shape[0], \
            "Размер X_train должен быть равен размеру y_train"
        assert n_iters >= 1

        def dJ_sgd(theta, X_b_i, y_i):
            return X_b_i * (X_b_i.dot(theta) - y_i) * 2.

        def sgd(X_b, y, initial_theta, n_iters=5, t0=5, t1=50):

            def learning_rate(t):
                return t0 / (t + t1)

            theta = initial_theta
            m = len(X_b)
            for i_iter in range(n_iters):
                indexes = np.random.permutation(m)
                X_b_new = X_b[indexes,:]
                y_new = y[indexes]
                for i in range(m):
                    gradient = dJ_sgd(theta, X_b_new[i], y_new[i])
                    theta = theta - learning_rate(i_iter * m + i) * gradient

            return theta

        X_b = np.hstack([np.ones((len(X_train), 1)), X_train])
        initial_theta = np.random.randn(X_b.shape[1])
        self._theta = sgd(X_b, y_train, initial_theta, n_iters, t0, t1)

        self.intercept_ = self._theta[0]
        self.coef_ = self._theta[1:]

        return self

    def predict(self, X_predict):
        assert self.intercept_ is not None and self.coef_ is not None, \
            "Нужно обучить модель перед использованием!"
        assert X_predict.shape[1] == len(self.coef_), \
            "Кол-во признаков в X_predict должно быть равно кол-ву признаков в X_train"

        X_b = np.hstack([np.ones((len(X_predict), 1)), X_predict])
        return X_b.dot(self._theta)

    def score(self, X_test, y_test):
        y_predict = self.predict(X_test)
        return r2_score(y_test, y_predict)

    def __repr__(self):
        return "RegressionSGD()"

Используем созданный класс для обучения модели регрессии на сгенерированных ранее данных:

In [ ]:
reg = RegressionSGD()
reg.fit(X, y, n_iters=2)
reg.coef_, reg.intercept_

In [ ]:
plt.scatter(x, y)
plot_x = np.linspace(-4, 4, 101)
plt.plot(plot_x, reg.predict(plot_x.reshape(-1,1)), c='r', lw=10);

### Полиномиальная регрессия

Полиномиальная регрессия может использоваться для решения задачи регрессии для нелинейных данных. В полиномиальной регрессии используется кривая линия, соответствующая полиному степени больше, чем 1. Например, пусть входные данные соответствуют зависимости $y=0.5 x^2+x+2$ (с нормальным шумом):

In [ ]:
x = np.random.uniform(-3, 3, size=100) # вектор
X = x.reshape(-1, 1)                   # матрица с одним стобцом 
y = 0.5 * x**2 + x + 2 + np.random.normal(0, 1, size=100)

plt.scatter(x, y);

In [ ]:
reg = RegressionSGD()
reg.fit(X, y, n_iters=2)
y_predict = reg.predict(X)

plt.scatter(x, y)
plt.plot(x, y_predict, color='r');

Подготовим для модели регрессии входные данные с двумя признаками – линейной и квадратичной зависимостью от независимой переменной:

In [ ]:
X2 = np.hstack([X, X**2]) # соединение массивов по горизонтали
X2.shape

Применим к построенным данным модель регрессии на основе SGD и нарисуем набор данных и линию регрессии (функция `np.argsort` возвращает индексы элементов в отсортированном массиве):

In [ ]:
reg2 = RegressionSGD()
reg2.fit(X2, y, n_iters=2000)
y_predict2 = reg2.predict(X2)

plt.scatter(x, y)
plt.plot(np.sort(x), y_predict2[np.argsort(x)], c='r', lw=3); 

In [ ]:
plt.plot(x, y_predict2, c='r');

Определенные коэффициенты регрессии и смещение близки к значениям, использованным при генерации данных:

In [ ]:
reg2.coef_, reg2.intercept_

### Полиномиальная регрессия при помощи TensorFlow

Полиномиальная регрессия также может быть реализована при помощи TensorFlow. Создадим простейшую нейронную сеть с одним слоем из одного нейрона и двумя входными нейронами:

In [ ]:
reg2_model = tf.keras.Sequential([
    tf.keras.Input(shape=(2,)),
    tf.keras.layers.Dense(units=1)
])

В такой нейронной сети будет всего три обучаемых параметра:

In [ ]:
reg2_model.summary()

При компиляции модели в обязательном порядке указывается функция потерь (`loss`), также может быть выбран оптимизатор с параметрами (`optimizer`), метрики для оценки обучения (`metrics`) и некоторые другие другие параметры. В качестве функции потерь и/или метрики могут быть, в частности,  выбраны среднеквадратичная ошибка (MSE) и средняя абсолютная ошибка (MAE). Коэффициента детерминации $R^2$ среди стандартных функций потерь и метрик нет, но он легко может быть вычислен непосредственно по показателю MSE и общей дисперсии целевой переменной.

In [ ]:
reg2_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

Обучаем нейронную сеть на полиномиальных зависимостях:

In [ ]:
history = reg2_model.fit(
    X2, y, 
    epochs=100,
    # уровень выводимой информации
    verbose=1,
    # проверка (валидация) на 20% обучающих данных
    validation_split = 0.2)

Метод `fit` возвращает объект `history`, в котором для задачи регрессии обычно есть ключи `'loss'` и `'val_loss'`. Можно визуализировать историю обучения при помощи следующей функции:

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.ylim([0, max(history.history['loss'])*0.5])
  plt.title('Функция потерь при обучении модели')
  plt.xlabel('Эпохи обучения')
  plt.ylabel('Функция потерь')
  plt.legend(['Обучающая выборка', 'Тестовая выборка'], loc='upper right')
  plt.grid(True)

In [ ]:
plot_loss(history)

При помощи обученной модели можно выполнить прогноз:

In [ ]:
y_predict_reg2 = reg2_model.predict(X2)

plt.scatter(x, y, label='набор данных')
plt.plot(np.sort(x), y_predict_reg2[np.argsort(x)], color='r', label='прогноз')
plt.legend(loc='upper left')
plt.grid();

### Нелинейная парная регрессия при помощи TensorFlow

Универсальная теорема аппроксимации утверждает, что любую непрерывную функцию можно с любой степенью точности аппроксимировать нейронной сетью с одним скрытым слоем с функцией активации сигмоида.

Построим и обучим нейронную сеть такого типа для рассматриваемого набора данных:

In [ ]:
uni_model = tf.keras.Sequential([
    tf.keras.Input(shape=(1,)),
    tf.keras.layers.Dense(units=512, activation='sigmoid'),
    tf.keras.layers.Dense(units=1)
])

In [ ]:
uni_model.summary()

In [ ]:
uni_model.compile(loss='mse', 
                  optimizer=tf.optimizers.Adam(learning_rate=0.1),
                  metrics=['mae'])

In [ ]:
X.shape

In [ ]:
history = uni_model.fit(
    X, y, 
    epochs=100,
    # уровень выводимой информации
    verbose=1,
    # проверка (валидация) на 20% обучающих данных
    validation_split = 0.2)

In [ ]:
plot_loss(history)

In [ ]:
y_predict_uni = uni_model.predict(X)

plt.scatter(x, y, label='набор данных')
plt.plot(np.sort(x), y_predict_uni[np.argsort(x)], color='r', label='прогноз')
plt.legend(loc='upper left')
plt.grid();

### Нелинейная множественная регрессия при помощи TensorFlow

При помощи нейронных сетей с нелинейными функциями активации нейронов можно успешно решать задачи нелинейной регрессии.

Загрузим из TesorFlow Datasets набор `howell` с демографическими данными жителей Калахари:

In [ ]:
ds = tfds.load("howell", split='train')
df = tfds.as_dataframe(ds)
df.head()

Изучим зависимость возраста от роста и веса:

In [ ]:
X = np.array(df[['height','weight']])
y = np.array(df[['age']]).reshape(-1)

In [ ]:
X.shape, y.shape

### Визуализация трехмерных данных

Для построения 3d графиков необходимо импортировать необходимые модули:

In [ ]:
from mpl_toolkits import mplot3d
# или from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d') # или ax = fig.add_subplot(111, projection='3d')

Для построения точечного графика можно использовать функцию `scatter()`, которой передаются три параметра для координат точек по осям X, Y и Z.

In [ ]:
fig = plt.figure(figsize=(12,10)) 
ax = plt.axes(projection='3d') 

xs = X[:,0]
ys = X[:,1]
zs = y

ax.scatter( xs, ys, zs, s=100 ) 
ax.set_xlabel('Рост') 
ax.set_ylabel('Вес') 
ax.set_zlabel('Возраст') 
ax.view_init( azim=-120, elev=25 )

### Глубокая нейронная сеть для задачи регрессии

Используем слой нормализации, адаптированный к обоим независимым признакам:

In [ ]:
feature_normalizer = tf.keras.layers.Normalization(axis=None,input_shape=(2,)) 
feature_normalizer.adapt(X)

Создадим нейронную сеть со слоем нормализации, четырьмя скрытыми плотными слоями с 64 нейронами и функцией активации ReLu и выходным слоем из одного нейрона:

In [ ]:
large_model = tf.keras.Sequential([
    feature_normalizer,
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

large_model.summary()

Скомпилируем модель, используя в качестве функции потерь  среднеквадратичную ошибку MSE с оптимизатором по умолчанию (RmsProp):

In [ ]:
large_model.compile(loss='mse')

Обучим модель на наборе данных из двух признаков:

In [ ]:
history = large_model.fit(
    X, y, 
    epochs=100,
    # уровень выводимой информации
    verbose=1,
    # проверка (валидация) на 30% обучающих данных
    validation_split = 0.3)

Кривые обучения в зависимости от эпохи обучения выглядят так:

In [ ]:
plot_loss(history)

Для визуализации прогнозируемых множественной регрессией значений воспользуемся функцией `plot_surface`. Потребуются определенные усилия по подготовке данных для `plot_surface`:

In [ ]:
n_plot = 51

x_plot = np.linspace(np.min(xs), np.max(xs), n_plot) 
y_plot = np.linspace(np.min(ys), np.max(ys), n_plot)

In [ ]:
x_mesh, y_mesh = np.meshgrid(x_plot, y_plot)
x_mesh.shape, y_mesh.shape

In [ ]:
x_plot2 = np.reshape(x_mesh, [n_plot**2,1])
y_plot2 = np.reshape(y_mesh, [n_plot**2,1])
xy_2 = np.hstack([x_plot2, y_plot2])
xy_2.shape

Теперь выполним прогнозирование при помощи обученной ранее модели, после чего вернемся к форме 51 на 51:

In [ ]:
z = large_model.predict(xy_2)
z.shape

In [ ]:
z_mesh = z.reshape((n_plot, n_plot))
z_mesh.shape

Функция `plot_surface` имеет большое число параметров, в частности:

* X, Y, Z : 2D массивы – данные для построения поверхности.
* rstride, cstride : int – параметры определяют величину шага, с которым будут браться элементы строки/столбца из переданных массивов.
* cmap: Colormap – цветовая карта для элементов поверхности.


In [ ]:
from matplotlib import cm

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_surface(x_mesh, y_mesh, z_mesh, \
       rstride=1, cstride=1, linewidth=0.05, cmap=cm.winter, antialiased=True, \
       edgecolors='gray') 
ax.scatter( xs, ys, zs, s=100, c='r' )

ax.set_xlabel('Рост', fontsize=14) 
ax.set_ylabel('Вес', fontsize=14)
ax.set_zlabel('Возраст', fontsize=14) 
ax.set_title('Демографические данные обитателей Калахари', fontsize=16)

ax.set_zlim(0., z_mesh.max())
ax.view_init(elev = 20, azim = 120)

__Кривые обучения__ — это графическое представление зависимости меры (показателя) качества обучения (по вертикальной оси) от определенного показателя модели обучения (по горизонтальной оси). Будем визуализировать в качестве качества модели показатели RMSE для части обучающей выборки и тестовой выборки в зависимости от количества точек в обучающей выборке.

Для разбиения на обучающую и тестовую выборки можно использовать функцию `train_test_split`:

In [ ]:
def train_test_split(X, y, test_ratio=0.2, seed=None):
    """возвращает X_train, X_test, y_train, y_test"""
    assert X.shape[0] == y.shape[0], \
        "Размер X должен быть равен размеру y"
    assert 0.0 <= test_ratio <= 1.0, \
        "Неверное значение test_ratio"

    if seed:
        np.random.seed(seed)

    shuffled_indexes = np.random.permutation(len(X))

    test_size = int(len(X) * test_ratio)
    test_indexes = shuffled_indexes[:test_size]
    train_indexes = shuffled_indexes[test_size:]

    X_train = X[train_indexes]
    y_train = y[train_indexes]

    X_test = X[test_indexes]
    y_test = y[test_indexes]

    return X_train, X_test, y_train, y_test

Разобьем массивы данных `X` и `y` на обучающие и тестовые данные:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 0.3)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
def my_rmse(y_test, y_predict):
    return np.sqrt(np.sum((y_predict - y_test)**2) / len(y_test))

In [ ]:
train_score = []
test_score = []
for i in range(11, 381, 10):
    large_model = tf.keras.Sequential([
        feature_normalizer,
#        tf.keras.layers.Dense(units=64, activation='relu'),
#        tf.keras.layers.Dense(units=64, activation='relu'),
#        tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=1)
    ])
    large_model.compile(loss='mse')
    large_model.fit(X_train[:i], y_train[:i], epochs=50, verbose=0)

    y_train_predict = large_model.predict(X_train[:i])
    train_score.append(my_rmse(y_train[:i], y_train_predict))
    
    y_test_predict = large_model.predict(X_test)
    test_score.append(my_rmse(y_test, y_test_predict))
    print('-->', i, ' done')

In [ ]:
plt.plot([i for i in range(11, len(X_train), 10)], 
                               train_score, label="train")
plt.plot([i for i in range(11, len(X_train), 10)], 
                               test_score, label="test")
plt.legend();

#### Задание (10 баллов)

Для закрепленного за Вами варианта лабораторной работы:

1.	Загрузите заданный в индивидуальном задании набор данных из Tensorflow Datasets, включая указанные в задании независимый признак и зависимый признак (отклик).

2.	Решите задачу полиномиальной регрессии для степени полинома, указанной в индивидуальном задании, при помощи нейронной сети с одним нейроном и оцените качество полученной модели по показателю, указанному в индивидуальном задании.   

3. Постройте кривые обучения с зависимостью от количества эпох.

4.  Визуализируйте точки набора данных на плоскости в виде диаграммы рассеяния (ось X – независимый признак, ось Y – зависимый признак), а также линию регрессии (другим цветом), подписывая оси и рисунок.  

5. Определите в исходном наборе данных признак (отличный от независимого и зависимого признаков), принимающий непрерывные значения и имеющий свойства, указанные в индивидуальном задании. 

6. Визуализируйте этот признак в соответствии с индивидуальным заданием. 

7. Сформируйте набор входных из двух признаков набора данных (независимый признак и определенный признак), создайте и адаптируйте нормализующий слой Tensorflow для двух признаков. 

8.	Используя созданный нормализующий слой, постройте нейронную сеть (нелинейный регресор) с количеством скрытых слоев, количеством нейронов и функцией активации, указанными в индивидуальном задании, и одним нейроном в выходном слое и обучите ее на наборе данных из двух признаков и отклика. 

9. Визуализируйте набор данных в виде точечного графика и прогноз нейронной сети в виде поверхности в трехмерном пространстве.

10. Разбейте набор данных из двух признаков и отклика на обучающую и тестовую выборки и постройте кривые обучения для заданного показателя качества в зависимости от количества точек в обучающей выборке, подписывая оси и рисунок и создавая легенду. 